<a href="https://colab.research.google.com/github/krozboy72/flightSearcher/blob/main/eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pywinauto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 16.6 MB/s eta 0:00:00


In [2]:
from dataclasses import dataclass, field
from typing import List, Optional
import base64

@dataclass
class InputFormat:
    prompt: str
    single_click: bool
    screen_interaction: bool
    screen_recording: bool

@dataclass
class OutputFormat:
    id: int
    short_caption: str
    describtion: str

@dataclass
class singleOutputFormat:
    control_name: str = ""
    action_type: str = ""
    short_caption: str = ""
    x_coordinate: int = 0
    y_coordinate: int = 0
    describtion: str = "This is the first step of the execution."
    text_to_type: str = ""
    scroll_direction: str = ""
    duration: int = 1000
    max_attempts: int = 1
    delay_between_attempts: int = 1
    priority: str = "High"
    next_step:str = ""
    last:bool = False
    skip:bool = False

@dataclass
class finalOutputFormat:
    step_number: int
    control_name: str
    action_type: str
    short_caption: str
    describtion: str
    x_coordinate: int
    y_coordinate: int
    text_to_type: str = ""
    scroll_direction: str = ""
    duration: int = 1000
    max_attempts: int = 2
    delay_between_attempts: int = 3
    priority: str = "Low"


@dataclass
class WebControls:
    buttons: list[str]
    hyperlinks: list[str]
    inputs: list[str]
    selects: list[str]

@dataclass
class Step:
    position: int
    image: base64
    describtion: str

@dataclass
class ui_element:
    name: str = ""
    control_type: str = ""
    hndw: int = None
    left: int = 0
    right: int = 0
    top: int = 0
    bottom: int = 0
    scroll: int = 0


    def __repr__(self):
        return f"ui_element(name='{self.name}', control_type='{self.control_type}')"

    def to_dict(self): # Nützlich für Vergleiche
        return {"name": self.name, "type": self.control_type}

@dataclass
class ScreenInf:
    windows: list[str] = field(default_factory=list)
    taskbar: list[ui_element] = field(default_factory=list)
    foreground_window: str = ""
    ui_controls: list[str] = None
    ui_elements: list[ui_element] = None
    web_elements: list[ui_element] = None

@dataclass
class trainOutputFormat:
    current_screen_info: ScreenInf
    step_number: int
    control_name: str
    control_type: str
    action_type: str
    short_caption: str
    describtion: str
    x_coordinate:int
    y_coordinate:int


In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.7 MB/s eta 0:00:00


In [4]:
!pip install sentence_transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
import sys
!{sys.executable} -m pip install json


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [6]:

import time
from sentence_transformers import SentenceTransformer
import faiss
from typing import List, Tuple, Optional
import numpy as np
import torch
from umap import UMAP
import matplotlib.pyplot as plt



class UIElementRetriever:
    def __init__(self, model_name: str = "intfloat/multilingual-e5-large") -> None:
        """
        Klasse zur Vektorisierung und FAISS-basierten Suche von UI-Elementen.
        Nutzt GPU (CUDA) via PyTorch und FAISS, falls verfügbar.
        """

        self.model = SentenceTransformer(model_name)
        self.ui_index: Optional[faiss.Index] = None
        self.web_index: Optional[faiss.Index] = None
        self.ui_elements: List[ui_element] = []
        self.web_elements: List[ui_element] = []
        self.ui_embeddings: Optional[np.ndarray] = None
        self.web_embeddings: Optional[np.ndarray] = None

    def _create_and_train_index(self, embeddings: np.ndarray, nlist: int = 64) -> faiss.Index:
        embeddings = embeddings.astype(np.float32)
        dim = embeddings.shape[1]
        n_samples = embeddings.shape[0]

        if n_samples < nlist:
            cpu_index = faiss.IndexFlatL2(dim)
            cpu_index.add(embeddings)
        else:
            quantizer = faiss.IndexFlatL2(dim)
            cpu_index = faiss.IndexIVFFlat(quantizer, dim, nlist)
            if not cpu_index.is_trained:
                cpu_index.train(embeddings)
            cpu_index.add(embeddings)

        if torch.cuda.is_available() and faiss.get_num_gpus() > 0:
            res = faiss.StandardGpuResources()
            gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
            return gpu_index
        else:
            return cpu_index

    def _deduplicate_elements(self, elements: List[ui_element]) -> List[ui_element]:
        seen = {}
        for el in elements:
            key = (el.name, el.control_type, getattr(el, 'automation_id', ''), getattr(el, 'id', ''))
            if key not in seen:
                seen[key] = el
        return list(seen.values())

    def build_index(self, screen: ScreenInf) -> None:
        device_str = "cuda" if torch.cuda.is_available() else "cpu"

        # UI-Elemente vorbereiten
        self.ui_elements = self._deduplicate_elements(screen.ui_elements or [])
        ui_texts = [
            f"{el.name} ({el.control_type}) {getattr(el, 'automation_id', '')} {getattr(el, 'id', '')}"
            for el in self.ui_elements
        ]

        if ui_texts:
            self.ui_embeddings = self.model.encode(
                ui_texts,
                normalize_embeddings=True,
                convert_to_numpy=True,
                batch_size=32,
                show_progress_bar=True,
                device=device_str
            )
            self.ui_index = self._create_and_train_index(self.ui_embeddings)

        # Web-Elemente vorbereiten
        self.web_elements = self._deduplicate_elements(screen.web_elements or [])
        web_texts = [
            f"{el.name} ({el.control_type}) {getattr(el, 'automation_id', '')} {getattr(el, 'id', '')}"
            for el in self.web_elements
        ]

        if web_texts:
            self.web_embeddings = self.model.encode(
                web_texts,
                normalize_embeddings=True,
                convert_to_numpy=True,
                batch_size=32,
                show_progress_bar=True,
                device=device_str
            )
            self.web_index = self._create_and_train_index(self.web_embeddings)

    def get_similar_elements(
            self,
            query: str,
            top_k_ui: int = 30,
            top_k_web: int = 30
    ) -> Tuple[List[ui_element], List[ui_element]]:
        if self.ui_index is None and self.web_index is None:
            raise ValueError("Index wurde noch nicht aufgebaut. Bitte zuerst build_index() aufrufen.")

        device_str = "cuda" if torch.cuda.is_available() else "cpu"
        query_embedding = self.model.encode(
            [query],
            normalize_embeddings=True,
            convert_to_numpy=True,
            device=device_str
        )

        ui_results_candidates = []
        if self.ui_index is not None and self.ui_elements: # Check if there are UI elements and an index
            # Determine the number of neighbors to search for, capped by available elements
            k_ui = min(top_k_ui, len(self.ui_elements))
            if k_ui > 0: # Proceed only if k_ui is positive
                distances, ui_indices = self.ui_index.search(query_embedding, k_ui)

                seen_indices = set() # To keep track of processed indices from the search results
                for idx in ui_indices[0]: # ui_indices[0] contains the indices from self.ui_elements
                    # Ensure index is valid (-1 can be returned by FAISS) and not already processed
                    if idx != -1 and idx not in seen_indices:
                        # Additional safety: check if index is within bounds of self.ui_elements
                        if 0 <= idx < len(self.ui_elements):
                            ui_results_candidates.append(self.ui_elements[idx])
                            seen_indices.add(idx)
                        # else: you could log an unexpected index value here if needed

        web_results_candidates = []
        if self.web_index is not None and self.web_elements: # Check if there are Web elements and an index
            # Determine the number of neighbors to search for, capped by available elements
            k_web = min(top_k_web, len(self.web_elements))
            if k_web > 0: # Proceed only if k_web is positive
                distances, web_indices = self.web_index.search(query_embedding, k_web)

                seen_indices = set() # To keep track of processed indices
                for idx in web_indices[0]:
                    if idx != -1 and idx not in seen_indices:
                        if 0 <= idx < len(self.web_elements):
                            web_results_candidates.append(self.web_elements[idx])
                            seen_indices.add(idx)
                        # else: you could log an unexpected index value here if needed

        # Deduplicate the collected lists using your existing _deduplicate_elements method
        final_ui_results = self._deduplicate_elements(ui_results_candidates)
        final_web_results = self._deduplicate_elements(web_results_candidates)

        return final_ui_results, final_web_results


    def plot_ui_umap(
        self,
        query: str,
        top_k: int = 30,
        title: str = "UMAP-Projektion der UI-Elemente"
    ) -> None:
        """
        Visualisiert die UI-Elemente in 2D mittels UMAP. Hebt die top_k relevanten UI-Elemente hervor.
        """
        retriever = self
        if retriever.ui_embeddings is None:
            raise ValueError("UI-Embeddings fehlen. Bitte zuerst build_index() ausführen.")

        # Führe RAG-Suche aus, um relevante UI-Elemente zu finden
        top_ui_elements, _ = retriever.get_similar_elements(query, top_k_ui=top_k, top_k_web=0)

        # Mapping von UI-Objekten zu Index
        ui_element_to_index = {self.get_element_key(el): idx for idx, el in enumerate(retriever.ui_elements)}
        selected_indices = [ui_element_to_index[self.get_element_key(el)] for el in top_ui_elements if self.get_element_key(el) in ui_element_to_index]
        # UMAP-Projektion
        reducer = UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
        embeddings_2d = reducer.fit_transform(retriever.ui_embeddings)

        # Scatterplot vorbereiten
        plt.figure(figsize=(10, 8))

        # Alle Punkte (grau)
        plt.scatter(
            embeddings_2d[:, 0],
            embeddings_2d[:, 1],
            c='lightgray',
            s=30,
            label='Alle UI-Elemente'
        )

        # Hervorhebung der 30 zurückgegebenen Punkte (rot)
        if selected_indices:
            plt.scatter(
                embeddings_2d[selected_indices, 0],
                embeddings_2d[selected_indices, 1],
                c='red',
                s=60,
                label=f"Top {top_k} Treffer zum Prompt",
                edgecolors='black'
            )

        # Plot-Stil
        plt.title(title)
        plt.xlabel("UMAP-Dimension 1")
        plt.ylabel("UMAP-Dimension 2")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def get_element_key(self, el):
        return (el.name, el.control_type, getattr(el, 'automation_id', ''), getattr(el, 'id', ''))

In [7]:
import json # <--- DIESE ZEILE IST WICHTIG
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from typing import List, Optional, Tuple, Dict, Any
from dataclasses import dataclass, field
import base64

import json

import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import random
from typing import List, Tuple, Optional, Any


try:
    import faiss
    FAISS_INDEX_TYPE = faiss.Index
except ImportError:
    print("Warnung: FAISS-Bibliothek nicht gefunden. FAISS-bezogene Funktionalität wird nicht funktionieren.")
    # Platzhaltertyp für faiss.Index definieren, falls faiss nicht installiert ist.
    class FaissIndexPlaceholder: pass
    FAISS_INDEX_TYPE = FaissIndexPlaceholder

class ModelTrainer:
    def __init__(self, dataset: List[Tuple[str, str, float]], base_model_name: str = "intfloat/multilingual-e5-base", results_file: str = "validation_results.txt"):
        """
        Klasse zum Trainieren eines SentenceTransformer-Modells und zur FAISS-basierten Indizierung.
        Nutzt GPU (CUDA) via PyTorch und FAISS, falls verfügbar.

        Args:
            dataset: Eine Liste von Tupeln, wobei jedes Tupel (text1, text2, score) ist.
                     'score' ist ein float-Wert (z.B. zwischen 0 und 1), der die Ähnlichkeit angibt.
            base_model_name: Name des zu verwendenden Basis-SentenceTransformer-Modells. Dieses Modell wird
                             vor jedem Trainingsinkrement neu geladen.
            results_file: Pfad zur Datei, in der die Validierungsergebnisse gespeichert werden.
        """
        self.base_model_name = base_model_name
        self.dataset = dataset  # List of (text1, text2, score)
        self.raw_train_data: List[Tuple[str, str, float]] = []
        self.raw_val_data: List[Tuple[str, str, float]] = []
        self.raw_test_data: List[Tuple[str, str, float]] = []

        self.val_evaluator: Optional[EmbeddingSimilarityEvaluator] = None
        self.results_file = results_file

        # Ergebnisdatei am Anfang einer neuen Trainingssitzung initialisieren/leeren
        with open(self.results_file, "w", encoding="utf-8") as f:
            f.write("Inkrementelle Trainings-Validierungsergebnisse\n")
            f.write("="*50 + "\n")

    def _split_dataset(self, train_ratio: float = 0.7, val_ratio: float = 0.15, test_ratio: float = 0.15, random_seed: int = 42):
        """
        Teilt den Gesamtdatensatz in Trainings-, Validierungs- und Testsets auf.
        """
        if not (abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-9): # Prüfen, ob Summe 1.0 ist
             raise ValueError("Die Summe von train_ratio, val_ratio und test_ratio muss 1.0 ergeben.")

        # Kopie des Datensatzes erstellen und mischen
        shuffled_dataset = list(self.dataset)
        random.Random(random_seed).shuffle(shuffled_dataset)

        n = len(shuffled_dataset)
        train_end = int(train_ratio * n)
        val_end = train_end + int(val_ratio * n)

        self.raw_train_data = shuffled_dataset[:train_end]
        self.raw_val_data = shuffled_dataset[train_end:val_end]
        self.raw_test_data = shuffled_dataset[val_end:]

        print(f"Datensatz aufgeteilt: {len(self.raw_train_data)} Trainingspaare, "
              f"{len(self.raw_val_data)} Validierungspaare, "
              f"{len(self.raw_test_data)} Testpaare.")

        if not self.raw_val_data:
            print("Warnung: Validierungsdatensatz ist leer. Validierung wird nicht möglich sein.")
            self.val_evaluator = None
        else:
            val_sentences1 = [data[0] for data in self.raw_val_data]
            val_sentences2 = [data[1] for data in self.raw_val_data]
            val_scores = [data[2] for data in self.raw_val_data]
            self.val_evaluator = EmbeddingSimilarityEvaluator(val_sentences1, val_sentences2, val_scores, name="sts-validation")

    def _create_and_train_faiss_index(self, embeddings: np.ndarray, nlist: int = 64) -> Optional[FAISS_INDEX_TYPE]:
        """
        Erstellt und trainiert einen FAISS-Index mit den gegebenen Embeddings.
        Nutzt GPU, falls verfügbar. Diese Methode ist eine Utility-Funktion.
        """
        if 'faiss' not in globals() or FAISS_INDEX_TYPE is FaissIndexPlaceholder:
            print("FAISS Bibliothek nicht geladen oder nicht funktionsfähig. Indexerstellung übersprungen.")
            return None

        embeddings = embeddings.astype(np.float32) # FAISS erwartet float32
        dim = embeddings.shape[1]
        n_samples = embeddings.shape[0]

        if n_samples == 0:
            print("Embeddings-Array ist leer. FAISS-Index kann nicht erstellt werden.")
            return None

        # Für IndexIVFFlat sollte n_samples idealerweise deutlich größer als nlist sein.
        # Eine Faustregel für die Trainingsanforderung von IndexIVFFlat ist z.B. n_samples > nlist * 39.
        # Hier verwenden wir eine einfachere Heuristik: Wenn zu wenige Samples vorhanden sind, wird IndexFlatL2 verwendet.
        # nlist: Anzahl der Voronoi-Zellen. Mindestens 1.
        safe_nlist = max(1, nlist)
        if n_samples < safe_nlist * 4 or n_samples < 256 : # Mindestanzahl für IVFFlat oder wenn n_samples nicht viel größer als nlist
            print(f"Verwende IndexFlatL2 für {n_samples} Samples (Dimension: {dim}).")
            cpu_index = faiss.IndexFlatL2(dim)
            cpu_index.add(embeddings)
        else:
            print(f"Verwende IndexIVFFlat für {n_samples} Samples (Dimension: {dim}, nlist: {safe_nlist}).")
            quantizer = faiss.IndexFlatL2(dim) # Quantizer für IndexIVFFlat
            cpu_index = faiss.IndexIVFFlat(quantizer, dim, safe_nlist, faiss.METRIC_L2) # METRIC_L2 für Euklidische Distanz
            if not cpu_index.is_trained:
                # FAISS erfordert, dass genügend Daten zum Trainieren des Index vorhanden sind (typischerweise mehr als nlist).
                if n_samples >= safe_nlist : # Einfache Prüfung; idealerweise deutlich mehr
                    cpu_index.train(embeddings)
                else:
                    print(f"Nicht genügend Samples ({n_samples}) zum Trainieren von IndexIVFFlat mit nlist={safe_nlist}. Fallback auf IndexFlatL2.")
                    cpu_index = faiss.IndexFlatL2(dim) # Fallback
            cpu_index.add(embeddings) # Hinzufügen der Embeddings nach dem Training (oder direkt bei FlatL2)

        if torch.cuda.is_available() and hasattr(faiss, 'StandardGpuResources') and faiss.get_num_gpus() > 0:
            try:
                print("Versuche, FAISS Index auf GPU zu erstellen...")
                res = faiss.StandardGpuResources()
                gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
                print("FAISS Index erfolgreich auf GPU erstellt.")
                return gpu_index
            except Exception as e:
                print(f"Fehler beim Erstellen des GPU-Index: {e}. Fallback auf CPU-Index.")
                return cpu_index
        else:
            print("Keine GPU für FAISS verfügbar oder FAISS GPU-Unterstützung nicht kompiliert. Verwende CPU-Index.")
            return cpu_index

    def train_and_validate_incrementally(self, max_pairs_increment: int = 100, epochs_per_increment: int = 10, batch_size: int = 16, learning_rate: float = 2e-5, warmup_steps_ratio: float = 0.1):
        """
        Trainiert das Modell inkrementell. Für jedes Inkrement wird das Basismodell neu geladen und
        auf der entsprechenden Menge an Trainingsdaten trainiert, dann validiert.
        """
        self._split_dataset() # Datensatz einmalig aufteilen

        if not self.raw_train_data:
            print("Trainingsdatensatz ist leer. Training wird abgebrochen.")
            self._save_validation_result("Fehler: Trainingsdatensatz ist leer.\n")
            return

        num_total_train_examples = len(self.raw_train_data)

        last_processed_size = -1 # Um sicherzustellen, dass die erste Größe (auch 0, falls leer) verarbeitet wird

        # Inkrementelle Trainingsschleife
        for step in range(1, (num_total_train_examples // max_pairs_increment) + 2):
            current_target_size_this_step = step * max_pairs_increment
            current_train_size = min(current_target_size_this_step, num_total_train_examples)

            if current_train_size <= last_processed_size and current_train_size < num_total_train_examples :
                # Überspringen, wenn sich die Größe nicht erhöht hat, es sei denn, es ist das letzte Inkrement (volle Datenmenge)
                continue

            if current_train_size == 0 and num_total_train_examples > 0: # Sollte nicht passieren, wenn Trainingsdaten vorhanden sind
                 continue


            print(f"\n--- Start: Training mit {current_train_size} Datenpaaren für {epochs_per_increment} Epochen ---")
            self._save_validation_result(f"\n--- Training mit {current_train_size} Datenpaaren, {epochs_per_increment} Epochen ---\n")

            # Lade das Basismodell für jeden Trainingslauf neu, um faire Vergleiche zu ermöglichen
            current_model = SentenceTransformer(self.base_model_name)

            # Aktuelles Subset der Trainingsdaten vorbereiten
            current_raw_training_subset = self.raw_train_data[:current_train_size]
            train_examples = [InputExample(texts=[data[0], data[1]], label=data[2]) for data in current_raw_training_subset]

            if not train_examples:
                print(f"Keine Trainingsbeispiele für {current_train_size} Paare. Überspringe.")
                self._save_validation_result("Keine Trainingsbeispiele in diesem Inkrement.\n")
                last_processed_size = current_train_size
                if current_train_size == num_total_train_examples: break
                continue

            train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
            train_loss = losses.CosineSimilarityLoss(model=current_model)

            num_training_steps = len(train_dataloader) * epochs_per_increment
            warmup_steps = int(num_training_steps * warmup_steps_ratio)

            # Modell trainieren
            current_model.fit(train_objectives=[(train_dataloader, train_loss)],
                              epochs=epochs_per_increment,
                              warmup_steps=warmup_steps,
                              optimizer_params={'lr': learning_rate},
                              output_path=None, # Kein Zwischenspeichern des Modells hier
                              show_progress_bar=True,
                              evaluator=None, # Validierung erfolgt manuell danach
                              evaluation_steps=0)

            # Validierung nach dem Training für das aktuelle Inkrement
            print(f"\n--- Validierung nach Training mit {current_train_size} Datenpaaren ---")
            if self.val_evaluator:
                validation_score = self.val_evaluator(current_model, output_path=None)
                result_string = f"Datenpaare: {current_train_size}, Validierungs-Score (Korrelation): {validation_score:.4f}\n"
            else:
                result_string = f"Datenpaare: {current_train_size}, Keine Validierung durchgeführt (Evaluator nicht konfiguriert oder Validierungsdaten leer).\n"

            print(result_string.strip())
            self._save_validation_result(result_string)

            last_processed_size = current_train_size
            # Wenn alle Trainingsdaten verarbeitet wurden, Schleife beenden
            if current_train_size == num_total_train_examples:
                break

        print("\n--- Inkrementelles Training und Validierung abgeschlossen. Ergebnisse in:", self.results_file, "---")
        # Optional: Das zuletzt trainierte Modell zurückgeben, um es z.B. auf dem Testset zu evaluieren
        # return current_model # current_model wäre hier das mit der vollen Trainingsmenge trainierte Modell

    def _save_validation_result(self, result_string: str):
        """
        Speichert das Validierungsergebnis in der Ergebnisdatei.
        """
        try:
            with open(self.results_file, "a", encoding="utf-8") as f:
                f.write(result_string)
        except IOError as e:
            print(f"Fehler beim Schreiben der Ergebnisdatei {self.results_file}: {e}")

    def evaluate_on_test_set(self, model_to_evaluate: SentenceTransformer, batch_size: int = 16):
        """
        Evaluiert ein gegebenes trainiertes Modell auf dem Testdatensatz.
        Das Modell sollte das Ergebnis des letzten (oder eines beliebigen) Trainingsinkrements sein.
        """
        if not self.raw_test_data:
            print("Testdatensatz ist leer. Evaluierung wird abgebrochen.")
            self._save_validation_result("\nTest-Evaluierung: Testdatensatz leer.\n")
            return None

        print("\n--- Evaluierung auf dem Testdatensatz ---")

        test_sentences1 = [data[0] for data in self.raw_test_data]
        test_sentences2 = [data[1] for data in self.raw_test_data]
        test_scores = [data[2] for data in self.raw_test_data]

        if not test_sentences1:
            print("Keine Daten im Testset für Evaluierung.")
            self._save_validation_result("\nTest-Evaluierung: Keine Testdaten vorhanden.\n")
            return None

        test_evaluator = EmbeddingSimilarityEvaluator(test_sentences1, test_sentences2, test_scores, name="sts-test", batch_size=batch_size)
        test_score = test_evaluator(model_to_evaluate, output_path=None)

        result_string = f"Endergebnis Test-Score (Korrelation): {test_score:.4f}\n"
        print(result_string.strip())
        self._save_validation_result(f"\n--- Test Set Evaluation ---\n{result_string}")
        return test_score

# # Beispielhafte Verwendung (nicht Teil der Klasse, nur zur Demonstration):
# if __name__ == '__main__':
#     # Erstellen eines Beispieldatensatzes mit 800 Paaren
#     # Format: (Satz1, Satz2, Ähnlichkeits-Score)
#     dummy_dataset = []
#     for i in range(800):
#         # Beispiel-Scores; in der Realität kämen diese aus Ihren Daten
#         score = random.uniform(0, 1)
#         dummy_dataset.append((f"Das ist Satz A Nummer {i}", f"Hier steht Satz B {i}", score))

#     print(f"Gesamter Datensatz enthält {len(dummy_dataset)} Paare.")

#     # Initialisieren des Trainers
#     trainer = ModelTrainer(dataset=dummy_dataset,
#                            base_model_name="sentence-transformers/paraphrase-MiniLM-L3-v2", # Kleineres Modell für schnelles Testen
#                            results_file="meine_trainingsergebnisse.txt")

#     trainer.train_and_validate_incrementally(max_pairs_increment=100,
#                                              epochs_per_increment=3, # Weniger Epochen für schnelles Testen
#                                              batch_size=8)

#     print("\nLade ein Modell für die Test-Evaluierung (hier nur das Basismodell als Beispiel):")
#     example_model_for_testing = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L3-v2")
#     trainer.evaluate_on_test_set(model_to_evaluate=example_model_for_testing)

#     print(f"\nAlle Vorgänge abgeschlossen. Ergebnisse sind in {trainer.results_file} und ggf. in der Konsole zu sehen.")



def main_test_retriever():
    input_jsonl_path = r"/content/converted_output_rag_prezess4.jsonl" # Ihre Eingabedatei
    TOP_K_RESULTS = 20
    found_count = 0
    processed_count = 0
    errors_count = 0
    verifiable_entries = 0

    retriever = UIElementRetriever()

    try:
        with open(input_jsonl_path, "r", encoding="utf-8") as f:
            for line_idx, line in enumerate(f):
                print(f"\n--- Processing Entry {line_idx + 1} ---")
                try:
                    data_entry = json.loads(line)
                    processed_count += 1

                    goal_query = data_entry.get("goal")
                    available_controls_data = data_entry.get("available_controls", [])
                    print(f"The number of controls RAG is choosing: {len(available_controls_data)}")
                    suggested_action_data = data_entry.get("suggested_action")

                    if not goal_query or not available_controls_data:
                        print("Skipping entry: 'goal' or 'available_controls' is missing/empty.")
                        errors_count +=1
                        continue

                    current_ui_elements = []
                    for ctrl_data in available_controls_data:
                        current_ui_elements.append(
                            ui_element(name=ctrl_data.get("name", ""), control_type=ctrl_data.get("type", ""))
                        )

                    if not current_ui_elements:
                        print("No UI elements to build index from after parsing. Skipping.")
                        errors_count +=1
                        continue

                    screen_info = ScreenInf(ui_elements=current_ui_elements)

                    # Index für diesen spezifischen Eintrag bauen
                    retriever.build_index(screen_info) # Baut ui_embeddings und ui_index neu auf

                    print(f"Querying with goal: '{goal_query}'")
                    similar_ui_elements, _ = retriever.get_similar_elements(query=goal_query, top_k_ui=TOP_K_RESULTS)
                    print("The right answer is:", suggested_action_data)



                    if suggested_action_data:
                        verifiable_entries += 1
                        is_found = False
                        suggested_name = suggested_action_data.get("name")
                        suggested_type = suggested_action_data.get("type")

                        for retrieved_el in similar_ui_elements:
                            if retrieved_el.name == suggested_name and retrieved_el.control_type == suggested_type:
                                is_found = True
                                break

                        if is_found:
                            found_count += 1
                            print(f"SUCCESS: Suggested action '{suggested_name}' ({suggested_type}) found.")
                        else:
                            print(f"FAILURE: Suggested action '{suggested_name}' ({suggested_type}) NOT found.")
                    else:
                        print("No 'suggested_action' to verify against for this entry.")

                except Exception as e:
                    print(f"Error processing entry {line_idx + 1}: {e}")
                    errors_count +=1

    except FileNotFoundError:
        print(f"Error: Input file '{input_jsonl_path}' not found.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    print("\n--- Evaluation Summary ---")
    print(f"Total entries attempted: {processed_count}")
    print(f"Entries successfully processed up to verification (if suggestion existed): {verifiable_entries}")
    print(f"Suggested actions found in top {TOP_K_RESULTS} results: {found_count}")
    if verifiable_entries > 0:
         accuracy = (found_count / verifiable_entries) * 100
         print(f"Success rate (for verifiable entries): {accuracy:.2f}%")
    else:
        print("Success rate: N/A (no entries were verifiable or processed successfully for verification)")
    print(f"Entries skipped or failed during processing: {errors_count}")

if __name__ == "__main__":
    main_test_retriever()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

Querying with goal: 'Klicke auf Öffnen, um ein vorhandenes Dokument in Word zu laden.'
The right answer is: {'name': "element_name: 'Öffnen', element_type: 'Button'"}
FAILURE: Suggested action 'element_name: 'Öffnen', element_type: 'Button'' (None) NOT found.

--- Processing Entry 218 ---
The number of controls RAG is choosing: 140


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Querying with goal: 'Um eine Datei zu öffnen, kannst du in Word auf Öffnen klicken.'
The right answer is: {'name': "element_name: 'Öffnen', element_type: 'Button'"}
FAILURE: Suggested action 'element_name: 'Öffnen', element_type: 'Button'' (None) NOT found.

--- Processing Entry 219 ---
The number of controls RAG is choosing: 140


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
import torch
print("GPU verfügbar:", torch.cuda.is_available())
print("Name der GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine")

GPU verfügbar: False
Name der GPU: Keine


In [5]:
!pip install -r /content/requirements.txt


  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached google_cloud_aiplatform-1.75.0-py2.py3-none-any.whl.metadata (31 kB)
  Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached playwright-1.49.1-py3-none-manylinux1_x86_64.whl.metadata (3.5 kB)
  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyGetWindow-0.0.9.tar.gz (9.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached pynput-1.7.7-py2.py3-none-any.whl.metadata (31 kB)
  Using cached PyQt6-6.8.0-cp39-abi3-manylinux_2_35_x86_64.whl.metadata (2.1 kB)
  Using cached pyttsx3-2.98-py3-none-any.whl.metadata (3.8 kB)
  Using cached pywinauto-0.6.8-py2.py3-none-any.whl.metadata 